【ハイパーパラメタ・チューニング】<br>
これまで機械学習の手法として、SVMやランダムフォレストを使って、分類や予測をやってきましたが、<br>
実は、SVMやランダムフォレストには、ハイパーパラメータという「手動」で設定する必要がある変数があります。<br>
前にやった時は、ハイパーパラメータをデフォルト値のまま実行したことになります。<br>
今回は、また再びタイタニックを使い、このハイパーパラメータを見ていきましょう。

問1<br>
機械学習アルゴリズムのハイパーパラメータがどういう役割か、<br>また実際にSVM・ランダムフォレストの主なハイパーパラメータにどういうものがあるか、<br>またデフォルトの値が何か、調べてください。

<br>
ハイパーパラメータとは、機械学習の実行による推論や予測では決定されない、<br>
プログラム実施者が任意に決定するパラメータを言う。なお、「パラメータ」<br>
は、機械学習モデルが学習の過程で自動的に最適化を行うものを指す。
<br>
<主なハイパーパラメータ><br>
1. SVM<br>
① kernel(デフォルト:rbf - radial basis function)<br>
カーネルは、SVMの演算を行う'関数'、他に、'linear','poly'等の<br>
選択肢がある<br>
② C(コスト、デフォルト:1.0 <-実数)<br>
SVMの損失関数の誤差の係数(誤分類を許容する程度)<br>
③ gamma(γ、デフォルト:1/(特徴量の数*分散) <- 実数)<br>
kernel が 'rbf'・'poly'・'sigmoid'の際のカーネル関数の係数<br>
2. ランダムフォレスト<br>
① n_estimators(デフォルト:100-整数)<br>
ランダムフォレストは、「決定木」に設定した条件で元の集合を2分割し<br>
さらに別れた先で再度2分割する、を繰り返すが、その決定木の数<br>
② max_depth(デフォルト:None)<br>
決定木のノードの深さ(=階層)<br>
③ max_features(デフォルト:特徴量nの平方根)<br>
決定木それぞれに対し、ランダムに指定する特徴量の数。

問2<br>
1. タイタニックの'train.csv'を読み込み、予測精度に影響が少ない特徴量を外し、<br>
データを学習用:テスト用=7:3に分割し、SVMの学習モデルを作成し、予測精度を求めてください。<br>
(C,gammaの値はデフォルトで)

In [27]:
import pandas as pd

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
cd /content/drive/MyDrive/AIStudy/git/exc_hyperpara_titanic

/content/drive/MyDrive/AIStudy/git/exc_hyperpara_titanic


In [30]:
df = pd.read_csv('./data/train.csv')

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [32]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [33]:
print(df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [34]:
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].fillna('S')
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

In [35]:
x = df.drop(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp','Parch','Ticket', 'Cabin'], axis=1)
y = df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.3,random_state=7)

In [36]:
clf = SVC()
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
acc = accuracy_score(pred, y_test)
print('accuracy score : {:.5f}' .format(acc))

accuracy score : 0.65299


In [37]:
# ハイパーパラメータの値
clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

2. C, gammaの値ををそれぞれ数通り組合せ、予測精度の変化を確認してください。<br>
※C(デフォルト:1.0) ∈ 実数 <br>
&emsp;gamma(デフォルト:1/(特徴量の数*分散)) ∈ 実数

参考
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [38]:
# Cとgammaについて値を変化させる。
C_elems = [0.1,1,10,100,1000]
gamma_elems = [0.001,0.01,0.1,1]

In [39]:
max_score = 0
min_score = 1

In [40]:
for C in C_elems:
  for gamma in gamma_elems:
    clf = SVC(C=C,gamma=gamma)
    clf.fit(x_train, y_train)
    pred = clf.predict(x_test)
    acc = accuracy_score(pred, y_test)
#    print('C =',C,'gamma =',gamma)
    print('accuracy score : {:.5f}' .format(acc))
    if acc > max_score:
      max_score = acc
      max_score_C = C
      max_score_gamma = gamma
    if acc < min_score:
      min_score = acc
      min_score_C = C
      min_score_gamma = gamma

accuracy score : 0.65299
accuracy score : 0.67537
accuracy score : 0.58209
accuracy score : 0.58955
accuracy score : 0.65672
accuracy score : 0.68657
accuracy score : 0.73134
accuracy score : 0.75746
accuracy score : 0.74254
accuracy score : 0.72015
accuracy score : 0.74627
accuracy score : 0.77612
accuracy score : 0.74254
accuracy score : 0.73881
accuracy score : 0.74254
accuracy score : 0.75746
accuracy score : 0.75000
accuracy score : 0.76119
accuracy score : 0.74254
accuracy score : 0.74254


In [41]:
# スコアが最高であったときと、最低であった時を表示する。
print('C =',max_score_C,'gamma =',max_score_gamma)
print('max score : {:.5f}'.format(max_score))
print('C =',min_score_C,'gamma =',min_score_gamma)
print('min score : {:.5f}'.format(min_score))

C = 10 gamma = 1
max score : 0.77612
C = 0.1 gamma = 0.1
max score : 0.58209
